# Get more fancy pants data
- go to the web factset 
- request it
- back to your sheet and run it (insert the name)


# Integrate it with the model
- join the data
- so update the model features


In [ ]:
https://app.snowflake.com/marketplace/listing/GZT0ZGCQ51UL/factset-tick-history

In [ ]:
snowpark group by stock/day,
max price min price mean price, rebuild ML model

In [18]:
import yfinance as yf
import pandas as pd

data_final = pd.DataFrame()
tickers = ["IBM", "AMZN", "FDS", "MSFT", "AAPL","META"]
for tick in tickers:
    data = yf.download("AAPL", start="2020-01-01", end="2024-01-01")
    data = data.reset_index()
    data['SYMBOL'] = tick
    data_final = pd.concat([data_final, data], ignore_index=True, sort=False)
data_final.columns = ["DATE", "OPEN", "HIGH", "LOW", "CLOSE", "ADJ_CLOSE", "VOLUME", "SYMBOL"]
data_final = data_final[["DATE", "OPEN", "HIGH", "LOW", "CLOSE", "SYMBOL"]]
print(data_final.head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

        DATE       OPEN       HIGH        LOW      CLOSE SYMBOL
0 2020-01-02  74.059998  75.150002  73.797501  75.087502    IBM
1 2020-01-03  74.287498  75.144997  74.125000  74.357498    IBM
2 2020-01-06  73.447502  74.989998  73.187500  74.949997    IBM
3 2020-01-07  74.959999  75.224998  74.370003  74.597504    IBM
4 2020-01-08  74.290001  76.110001  74.290001  75.797501    IBM


In [19]:
data_final.to_csv("test.csv")

In [6]:
data['Date'].min()

Timestamp('2020-01-02 00:00:00')

In [ ]:
"IBM", "AMZN", "FDS", "MSFT", "AAPL","META"